In [1]:
import dask
# Avoids conflicts with Client for dask
from pystac_client import Client as clt
import planetary_computer
import leafmap
import stackstac
import xarray as xr

In [2]:
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 7.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37207,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 7.64 GiB
Comm: tcp://127.0.0.1:36433,Total threads: 4
Dashboard: http://127.0.0.1:40317/status,Memory: 1.91 GiB
Nanny: tcp://127.0.0.1:40117,


In [5]:
bbox = (32.16033157094111, 22.852984440390316, 33.288140399555346, 23.806193394664234)
#  Jan 1st, 2017 and Dec 31st, 2023
datetime = "2017-01-01/2024-12-31"

In [5]:
# def main(bbox, startDate, endDate):
#     def search_stac():

#     def return_assets(collection, asset_list, bbox):

#     def mean_ndwi(xarray_bands):

#     def plot_ndwi(ndwi_mean, date_time):

In [8]:
def search_stac():
    catalog = clt.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )
    
    search = catalog.search(
    collections=['sentinel-2-l2a'],
    bbox=bbox,
    )
    
    items = search.get_all_items()
    return items
    
    # items = list(search.get_all_items())  # Convert ItemCollection to a list of pystac.Item objects
    # return items

In [9]:
type(search_stac())

pystac.item_collection.ItemCollection

In [10]:
def return_assets(items, asset_list, bbox):
    # Create xarray object
    stack = stackstac.stack(items, assets = asset_list, bounds_latlon=bbox)
    return stack

In [9]:
xarray_object = return_assets(search_stac(), ["B03", "B08"], bbox)

In [10]:
type(xarray_object)

xarray.core.dataarray.DataArray

In [11]:
print(xarray_object)

<xarray.DataArray 'stackstac-0023caf00bfed9710e1b266582e20707' (time: 2348,
                                                                band: 2,
                                                                y: 10576,
                                                                x: 11572)> Size: 5TB
dask.array<fetch_raster_window, shape=(2348, 2, 10576, 11572), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 19kB 2015-...
    id                                       (time) <U54 507kB 'S2A_MSIL2A_20...
  * band                                     (band) <U3 24B 'B03' 'B08'
  * x                                        (x) float64 93kB 4.138e+05 ... 5...
  * y                                        (y) float64 85kB 2.633e+06 ... 2...
    s2:snow_ice_percentage                   float64 8B 0.0
    ...                                       ...
    gsd                         

In [12]:
xarray_object.band

<xarray.DataArray 'band' (band: 2)> Size: 24B
array(['B03', 'B08'], dtype='<U3')
Coordinates: (12/16)
  * band                                     (band) <U3 24B 'B03' 'B08'
    s2:snow_ice_percentage                   float64 8B 0.0
    instruments                              <U3 12B 'msi'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    s2:product_type                          <U7 28B 'S2MSI2A'
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    gsd                                      float64 8B 10.0
    proj:shape                               object 8B {10980}
    common_name                              (band) <U5 40B 'green' 'nir'
    center_wavelength                        (band) float64 16B 0.56 0.842
    full_width_half_max                      (band) float64 16B 0.045 0.145
    epsg                                     int64 8B 32636

In [18]:
def mean_ndwi(xarray_bands):
    # Get bands
    green = xarray_bands.sel(band = "B03")
    nir = xarray_bands.sel(band = "B08")

    ndwi = (green - nir) / (green + nir)

    # Add dimension argument to calculate over space (for each scene)
    mean_ndwi = ndwi.mean(dim=["x", "y"])

    return mean_ndwi

In [19]:
mean_ndwi = mean_ndwi(xarray_object)

In [21]:
type(mean_ndwi)

xarray.core.dataarray.DataArray

In [31]:
print(mean_ndwi)

<xarray.DataArray 'stackstac-0023caf00bfed9710e1b266582e20707' (time: 2348)> Size: 19kB
dask.array<mean_agg-aggregate, shape=(2348,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates: (12/37)
  * time                                     (time) datetime64[ns] 19kB 2015-...
    id                                       (time) <U54 507kB 'S2A_MSIL2A_20...
    s2:snow_ice_percentage                   float64 8B 0.0
    s2:not_vegetated_percentage              (time) float64 19kB 99.47 ... 70.25
    s2:water_percentage                      (time) float64 19kB 0.08308 ... ...
    s2:product_uri                           (time) <U65 610kB 'S2A_MSIL2A_20...
    ...                                       ...
    s2:reflectance_conversion_factor         (time) float64 19kB 1.026 ... 1.011
    s2:vegetation_percentage                 (time) float64 19kB 0.1457 ... 0...
    s2:unclassified_percentage               (time) float64 19kB 0.02922 ... ...
    gsd                        

In [36]:
def mean_ndwi_plot(mean_ndwi, start_date, end_date):
    # Date range
    date_range = pd.date_range(start = start_date, end = end_date)

    # Filter calculated NDWI per scene to date range
    ndwi_date_range = mean_ndwi.sel(time = date_range)

    # Extract values and dates
    ndwi_values = ndwi_date_range.values
    ndwi_dates = ndwi_date_range['time'].values
    
    plt.figure(figsize(15,10))
    plt.plot(ndwi_dates, ndwi_values)
    plt.xlabel("Time")
    plt.ylabel("Mean NDWI")
    plt.title("Mean NDWI Over Time")
    plt.legend()
    plt.show()

In [ ]:
mean_ndwi_plot(mean_ndwi_unique, "2017-01-01", "2024-12-31")